In [1]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, GlobalAveragePooling2D
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers import BatchNormalization
from keras.models import Model
from keras.layers import Dropout

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
img_width, img_height = 150, 150
batch_size = 32
epochs = 5


train_data_dir = '/content/drive/MyDrive/Final year project/CNN_test/Cropped_Images/train'
test_data_dir = '/content/drive/MyDrive/Final year project/CNN_test/Cropped_Images/test'

In [4]:
from keras.applications import vgg16

#base_model = tf.keras.applications.MobileNetV2(input_shape=(150, 150, 3),include_top=False,weights='imagenet')

base_model = vgg16.VGG16(weights='imagenet', include_top=False, input_shape=(150, 150, 3))

for layer in base_model.layers:
    layer.trainable = False

58900480/58889256 [==============================] - 0s 0us/step


In [5]:
def addTopOnNet(bottom_model, num_classes):
    top_model = bottom_model.output
    top_model = GlobalAveragePooling2D()(top_model)
    top_model = Dense(64,activation='relu')(top_model)
    top_model = Dropout(0.4)(top_model)
    top_model = Dense(32,activation='relu')(top_model)
    top_model = Dropout(0.4)(top_model)
    top_model = Dense(16,activation='relu')(top_model)
    top_model = Dropout(0.4)(top_model)
    top_model = Dense(num_classes,activation='sigmoid')(top_model)
    return top_model

In [6]:
FC_Head = addTopOnNet(base_model, 1)

model = Model(inputs = base_model.input, outputs = FC_Head)

print(model.summary())

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 150, 150, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 150, 150, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 75, 75, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 75, 75, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 75, 75, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 37, 37, 128)       0     

In [7]:
train_datagen = ImageDataGenerator(rescale=1. / 255)


test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

Found 3951 images belonging to 2 classes.
Found 997 images belonging to 2 classes.


In [8]:
nb_train_samples = 3951
nb_test_samples = 997

In [9]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])

In [10]:
model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_test_samples // batch_size)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  


Epoch 1/5
123/123 [==============================] - 1305s 10s/step - loss: 0.6288 - accuracy: 0.6234 - precision: 0.6106 - recall: 0.6828 - val_loss: 0.3938 - val_accuracy: 0.9466 - val_precision: 1.0000 - val_recall: 0.8936
Epoch 2/5
123/123 [==============================] - 27s 219ms/step - loss: 0.3115 - accuracy: 0.8798 - precision: 0.8820 - recall: 0.8770 - val_loss: 0.1586 - val_accuracy: 0.9435 - val_precision: 0.9911 - val_recall: 0.8954
Epoch 3/5
123/123 [==============================] - 27s 217ms/step - loss: 0.1683 - accuracy: 0.9546 - precision: 0.9650 - recall: 0.9434 - val_loss: 0.2745 - val_accuracy: 0.8851 - val_precision: 0.9974 - val_recall: 0.7731
Epoch 4/5
123/123 [==============================] - 27s 218ms/step - loss: 0.1006 - accuracy: 0.9730 - precision: 0.9793 - recall: 0.9663 - val_loss: 0.1230 - val_accuracy: 0.9597 - val_precision: 0.9914 - val_recall: 0.9276
Epoch 5/5
123/123 [==============================] - 27s 218ms/step - loss: 0.0763 - accuracy: 0

In [10]:
# for layer in model.layers:
#     print(layer)

for layer in model.layers[-12:]:
    layer.trainable = True

In [11]:
model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_test_samples // batch_size)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  


Epoch 1/5
123/123 [==============================] - 616s 5s/step - loss: 0.5786 - accuracy: 0.6846 - precision: 0.7089 - recall: 0.6256 - val_loss: 0.3432 - val_accuracy: 0.9143 - val_precision: 1.0000 - val_recall: 0.8293
Epoch 2/5
123/123 [==============================] - 18s 145ms/step - loss: 0.2634 - accuracy: 0.9061 - precision: 0.9299 - recall: 0.8788 - val_loss: 0.2003 - val_accuracy: 0.9244 - val_precision: 0.9953 - val_recall: 0.8534
Epoch 3/5
123/123 [==============================] - 18s 143ms/step - loss: 0.1545 - accuracy: 0.9515 - precision: 0.9668 - recall: 0.9353 - val_loss: 0.2379 - val_accuracy: 0.9083 - val_precision: 0.9975 - val_recall: 0.8189
Epoch 4/5
123/123 [==============================] - 18s 143ms/step - loss: 0.1057 - accuracy: 0.9712 - precision: 0.9818 - recall: 0.9602 - val_loss: 0.1269 - val_accuracy: 0.9446 - val_precision: 0.9385 - val_recall: 0.9517
Epoch 5/5
123/123 [==============================] - 18s 143ms/step - loss: 0.0866 - accuracy: 0.9

In [13]:
for layer in model.layers[-16:]:
    layer.trainable = True

In [14]:
model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_test_samples // batch_size)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  


Epoch 1/5
123/123 [==============================] - 18s 143ms/step - loss: 0.0649 - accuracy: 0.9832 - precision: 0.9881 - recall: 0.9781 - val_loss: 0.1760 - val_accuracy: 0.9435 - val_precision: 0.9933 - val_recall: 0.8936
Epoch 2/5
123/123 [==============================] - 18s 143ms/step - loss: 0.0489 - accuracy: 0.9862 - precision: 0.9892 - recall: 0.9832 - val_loss: 0.6125 - val_accuracy: 0.8256 - val_precision: 1.0000 - val_recall: 0.6519
Epoch 3/5
123/123 [==============================] - 18s 144ms/step - loss: 0.0421 - accuracy: 0.9900 - precision: 0.9933 - recall: 0.9867 - val_loss: 0.5032 - val_accuracy: 0.8589 - val_precision: 0.9972 - val_recall: 0.7209
Epoch 4/5
123/123 [==============================] - 18s 143ms/step - loss: 0.0365 - accuracy: 0.9911 - precision: 0.9923 - recall: 0.9898 - val_loss: 0.2286 - val_accuracy: 0.9385 - val_precision: 0.9977 - val_recall: 0.8793
Epoch 5/5
123/123 [==============================] - 17s 141ms/step - loss: 0.0368 - accuracy: 0